In [1]:
import sys
import pathlib
from IPython.display import Audio
from typing import List

import librosa
import numpy as np
import pandas as pd
from langdetect import detect
import deepcut
from string import punctuation
import re

ROOT_DIR = pathlib.Path.cwd().parent
sys.path.insert(0, str(ROOT_DIR))
from src.normalizers import EnglishTextNormalizer
from src.evaluate_utils import isd
from src.post_processer import remove_longest_repeating_words

2023-11-15 11:53:42.989956: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-15 11:53:43.391229: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 11:53:43.391353: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 11:53:43.391777: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 11:53:43.547685: I tensorflow/core/platform/cpu_feature_g

In [2]:
target_folder = ROOT_DIR / 'data/result/Thonburian-whisper-deepbrain-lang' / 'th_Oppday_Q2_2023_IP_interfama'
label_dir = target_folder / 'Groundtruth.csv'
th_result_dir = target_folder / 'th_result.csv'
en_result_dir = target_folder / 'en_result.csv'

In [3]:
label_df = pd.read_csv(str(label_dir))
th_result_df = pd.read_csv(str(th_result_dir)) 
en_result_df = pd.read_csv(str(en_result_dir))

In [4]:
merge_result_df = pd.concat(
    [th_result_df, en_result_df],
    axis=0
)
merge_result_df.shape

(888, 3)

In [5]:
merge_result_df.to_csv('./merge_result.csv')

In [6]:
sorted_result_df = merge_result_df.sort_values('start')

In [7]:
sorted_result_df.to_csv(str(target_folder / 'en-th_result.csv'))

In [8]:
prediction = sorted_result_df['text'].to_list()
references = label_df['text'].to_list()

In [9]:
text = "Q1 "
re.sub("[QA]:","", text)

'Q1 '

In [10]:
def en_preprocess(text: str) -> str:
    text = re.sub("[QA]:|(uh)","", text)
    return text

def th_preprocess(text: str) -> str:
    text = re.sub("((ค่ะ)|((นะ)?ครับ)|(นะคะ)|เอ่อ)+", "", text)
    # text = thaispellcheck.check(text, autocorrect=True)
    return text

def pred_postprocess(listtext: List[str]) -> List[str]:

    if " " in listtext:
        listtext.remove(" ")
        
    clean_text = remove_longest_repeating_words(listtext)
    return clean_text

def tokenize_text(text: str, pred = False) -> list:
    if not isinstance(text, str):
        return []
    try:
        lang = detect(text)
    except:
        lang = 'en'
    if lang != 'en':
        text = th_preprocess(text)
        splited_text = deepcut.tokenize(text)
    else:
        eng_normalizer = EnglishTextNormalizer()
        text = eng_normalizer(text)
        text = en_preprocess(text)
        splited_text = text.split(" ")
        
    return pred_postprocess(splited_text) if pred else splited_text

In [11]:
from functools import partial
prediction_words = list(map(partial(tokenize_text, pred=True), prediction))
references_words = list(map(tokenize_text, references))

2023-11-15 11:53:50.974790: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


2/2 [==============================] - 0s 19ms/step
{'วัน': 2, 'นี้': 1, 'ก็': 1, 'เป็น': 1, 'Auxility': 1, ' ': 2, 'Day': 1, 'ที่': 1, 'ผม': 1, 'จะ': 1, 'รายงาน': 1}
2/2 [==============================] - 0s 9ms/step
{'ทั้ง': 1, 'ของ': 2, 'คน': 1, 'และ': 1, 'สัตว์': 1, 'จาก': 1, 'ต้น': 1, 'น้ำ': 2, ' ': 1, 'เดิน': 1, 'ตาม': 1, 'ตอน': 1}
2/2 [==============================] - 0s 13ms/step
{'ก็': 2, 'ขอ': 1, 'มอบ': 1, 'บิน': 1, 'ทรัลสุขภาพ': 1, 'แบบ': 1, 'พวก': 1, 'องค์กร': 1}
1/1 [==============================] - 0s 51ms/step
{'จะ': 1, 'พูด': 1, 'ให้': 1, 'ฟัง': 1, 'ว่า': 2, 'เป็น': 1, 'อย่าง': 1, 'ไร': 1}
2/2 [==============================] - 0s 16ms/step
{'แล้ว': 1, 'ก็': 1, 'จาก': 1, 'ส้น': 1, 'น้ำ': 2, 'เป็น': 2, 'ปลาย': 1, 'อย่าง': 1, 'ไร': 1}
1/1 [==============================] - 0s 35ms/step
{'การ': 2, 'รักษา': 1, 'แล้ว': 1, 'ก็': 1, 'ป้องกัน': 1, 'โรค': 1}
2/2 [==============================] - 0s 17ms/step
{'แล้ว': 1, 'ก็': 2, 'ถัด': 1, 'มา': 1, 'อัน': 1, 'นี้': 1, 'จะ': 1,

In [12]:
from functools import reduce

prediction_words_reduce = list(reduce(lambda a, b: a + b, prediction_words))
prediction_words_reduce = [word for word in prediction_words_reduce if not word in ["", " "]]
references_words_reduce = list(reduce(lambda a, b: a + b, references_words))
references_words_reduce = [word for word in references_words_reduce if not word in ["", " "]]

In [13]:
result, step, summary = isd(prediction_words_reduce, references_words_reduce, debug=True) 
print(f"WER: {result / len(references_words_reduce)}")

REF: สวัสดี สวัสดี นัก ลง ทุน ทุก ท่าน แล้ว ก็  คุณ ผู้ ชม ทุก ท่าน วัน นี้ ก็ เป็น วัน Auxility    Day ที่ ผม จะ รายงาน           การ ล่าสุด ซึ่ง ก็ ล่าสุด ใน ชื่อ ไทม่าส์ 2 แล้ว ก็ อัปเดต เรื่อง ต่างๆ ของ บริษัท Interfarm มา เดี๋ยว โทษ ที  ก็                      วิสัยทัศน์ ของ บริษัท ก็ เรา ต้องการ เป็น พื้นฐาน ผลิตภัณฑ์ สุขภาพ แบบ ครบ วงจร นะ ทั้ง ของ คน และ     ของ สัตว์ จาก ต้น น้ำ เดิน ตาม    ตอน  น้ำ ก็ ขอ มอบ บิน       ทรัลสุขภาพ แบบ พวก องค์กร ก็ จะ พูด ให้ ฟัง ว่า ว่า เป็น อย่าง ไร แล้ว ก็ จาก ส้น น้ำ เป็น   ปลาย น้ำ เป็น อย่าง ไร  สำหรับ     ครบ วงจร เคย พลัด      สุขภาพ ใน    วิสัย ทาง        ของ บริษัท ก็ คือ ว่า จะ  หมาย ความ ว่า ตั้งแต่ ที่ หัว คน เดิม นิด        เลย ก็ คือ เรื่อง     เกี่ยว กับ นวัตกรรม ตอน นี้ งาน ของ ผม ที่ ทำ หลัก ๆ ก็ คือ เป็น การ ไป หา  อนุมัติธรรม ใหม่ ๆ ใน การ รักษา แล้ว ก็ การ ป้องกัน โรค แล้ว ก็ ถัด มา อัน นี้   ก็ จะ เป็น เรื่อง ของ ยา ซึ่ง ปัจจุบัน เรา มี โรง งาน ยา 2 โรง ที่ ผลิต ยา ที่ เรียก ว่า เจินดาลิก หรือ Phenocytical   ที่ เป็น เจินด

In [14]:
summary

,wer,insertions,deletions,substitutions,inserted_words,deleted_words,substituted_words
0,0.428784,327,1071,1542,"[ผม, ผม, มัน, แต่, ของ, ราคา, อะไร, ดู, อื่น, ...","[ได้, เปิด, ซับซ้าย, แบบ, ที่, ที่, ที่, เป็น,...","[(การ, กลาง), (ปี, ตี), (กษัตริย์, บริษัท), (เ..."


In [15]:
summary.to_excel("summary.xlsx",
             sheet_name='summary')
pd.DataFrame(summary.inserted_words).to_excel("summary.xlsx",
             sheet_name='inserted_words')
pd.DataFrame(summary.deleted_words).to_excel("summary.xlsx",
             sheet_name='deleted_words')
pd.DataFrame(summary.substituted_words).to_excel("summary.xlsx",
             sheet_name='substituted_words')

# Error Analysis

In [16]:
prediction_words_reduce

['สวัสดี',
 'สวัสดี',
 'นัก',
 'ลง',
 'ทุน',
 'ทุก',
 'ท่าน',
 'แล้ว',
 'ก็',
 'คุณ',
 'ผู้',
 'ชม',
 'ทุก',
 'ท่าน',
 'วัน',
 'นี้',
 'ก็',
 'เป็น',
 'วัน',
 'Auxility',
 'Day',
 'ที่',
 'ผม',
 'จะ',
 'รายงาน',
 'การ',
 'ล่าสุด',
 'ซึ่ง',
 'ก็',
 'ล่าสุด',
 'ใน',
 'ชื่อ',
 'ไทม่าส์',
 '2',
 'แล้ว',
 'ก็',
 'อัปเดต',
 'เรื่อง',
 'ต่างๆ',
 'ของ',
 'บริษัท Interfarm',
 'มา',
 'เดี๋ยว',
 'โทษ',
 'ที',
 'ก็',
 'วิสัยทัศน์',
 'ของ',
 'บริษัท',
 'ก็',
 'เรา',
 'ต้องการ',
 'เป็น',
 'พื้นฐาน',
 'ผลิตภัณฑ์',
 'สุขภาพ',
 'แบบ',
 'ครบ',
 'วงจร',
 'นะ',
 'ทั้ง',
 'ของ',
 'คน',
 'และ',
 'ของ',
 'สัตว์',
 'จาก',
 'ต้น',
 'น้ำ',
 'เดิน',
 'ตาม',
 'ตอน',
 'น้ำ',
 'ก็',
 'ขอ',
 'มอบ',
 'บิน',
 'ทรัลสุขภาพ',
 'แบบ',
 'พวก',
 'องค์กร',
 'ก็',
 'จะ',
 'พูด',
 'ให้',
 'ฟัง',
 'ว่า',
 'ว่า',
 'เป็น',
 'อย่าง',
 'ไร',
 'แล้ว',
 'ก็',
 'จาก',
 'ส้น',
 'น้ำ',
 'เป็น',
 'ปลาย',
 'น้ำ',
 'เป็น',
 'อย่าง',
 'ไร',
 'สำหรับ',
 'ครบ',
 'วงจร',
 'เคย',
 'พลัด',
 'สุขภาพ',
 'ใน',
 'วิสัย',
 'ทาง',
 'ของ',
 'บริษัท',
 

In [17]:
references_words_reduce

['สวัสดี',
 'นัก',
 'ลง',
 'ทุน',
 'ทุก',
 'ท่าน',
 'และ',
 'คุณ',
 'ผู้',
 'ชม',
 'ทุก',
 'ท่าน',
 'วัน',
 'นี้',
 'ก็',
 'เป็น',
 'วัน',
 'Opportunity',
 'Day',
 'ที่',
 'ผม',
 'จะ',
 'รายงาน',
 'ผล',
 'ประกอบ',
 'การ',
 'ล่าสุด',
 'ซึ่ง',
 'ล่าสุด',
 'ก็',
 'คือ',
 'ไตรมาส',
 '2',
 'แล้ว',
 'ก็',
 'อัปเดต',
 'เรื่อง',
 'ต่าง',
 'ๆ',
 'ของ',
 'บริษัท อินเตอร์ ฟาร์มา\n',
 'วิสัยทัศน์',
 'ของ',
 'บริษัท',
 'ก็',
 'เรา',
 'ต้องการ',
 'เป็น',
 'ผู้',
 'นำ',
 'สุขภาพ',
 'แบบ',
 'ครบ',
 'วงจร',
 'ทั้ง',
 'ของ',
 'คน',
 'แล้ว',
 'ก็',
 'ของ',
 'สัตว์',
 'จาก',
 'ต้น',
 'น้ำยัน',
 'ปลาย',
 'น้ำ',
 'ก็',
 'คำ',
 'ว่า',
 'ผลิตภัณฑ์',
 'สุขภาพ',
 'แบบ',
 'ครบ',
 'วงจร',
 'ก็',
 'จะ',
 'พูด',
 'ให้',
 'ฟัง',
 'ว่า',
 'เป็น',
 'อย่าง',
 'ไร',
 'แล้ว',
 'ก็',
 'จาก',
 'ต้น',
 'น้ำยัน',
 'ปลาย',
 'น้ำ',
 'เป็น',
 'อย่าง',
 'ไร\n',
 'คำ',
 'ว่า',
 'ครบ',
 'วงจร',
 'ผลิตภัณฑ์',
 'สุขภาพ',
 'เนี่ย',
 'ใน',
 'วิสัยทัศน์',
 'ของ',
 'บริษัท',
 'ก็',
 'คือ',
 'ว่า',
 'มัน',
 'หมาย',
 'ความ',
 'ว่า',
 'ตั้

In [18]:
import jiwer
jiwer.cer(hypothesis="".join(prediction_words_reduce), reference="".join(references_words_reduce))

0.36578666885299665